# 1. Load Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
answers_data = pd.read_csv('/kaggle/input/stacksample/Answers.csv', encoding = "ISO-8859-1")
questions_data = pd.read_csv('/kaggle/input/stacksample/Questions.csv', encoding = "ISO-8859-1")
tags_data = pd.read_csv('/kaggle/input/stacksample/Tags.csv', encoding = "ISO-8859-1")

In [ ]:
print(answers_data.shape)
answers_data.head()

In [ ]:
print(questions_data.shape)
questions_data.head()

In [ ]:
print(tags_data.shape)
tags_data.head()

In [ ]:
answers_data.ParentId.unique().shape[0]

In [ ]:
questions_data.Id.unique().shape[0]

In [ ]:
tags_data.Id.unique().shape[0]

In [ ]:
len(tags_data.Tag.unique())

In [ ]:
a = tags_data.Id.unique()-questions_data.Id.unique()
np.count_nonzero(a)

In [ ]:
b = questions_data.Id.unique()
c = answers_data.ParentId.unique()
questions_ids_with_no_answer = b[~np.in1d(b,c)]
questions_ids_with_no_answer.shape[0]

In [ ]:
mask_questions = questions_data["Id"].isin(questions_ids_with_no_answer)
questions_data = questions_data.loc[~mask_questions]

In [ ]:
mask_tags = tags_data["Id"].isin(questions_ids_with_no_answer)
tags_data = tags_data.loc[~mask_tags]

In [ ]:
questions_data.isnull().sum()

In [ ]:
answers_data.isnull().sum()

In [ ]:
tags_data.isnull().sum()

In [ ]:
tags_data = tags_data.dropna(axis=0)

In [ ]:
print(questions_data.shape)
print(answers_data.shape)
print(tags_data.shape)

# 2. Data Visualisation

## 2.1. Tags data grouped by tags name

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
tags=tags_data.Tag.value_counts().rename_axis('Tag').reset_index(name='counts')

In [ ]:
tags.head()

In [ ]:
tags = tags.loc[tags['counts']>100]

In [ ]:
print(tags.shape)
tags.head()

In [ ]:
plt.figure(figsize=(16,6))

plt.title("Questions Distribution According To Tags")
sns.barplot(x=tags.loc[:50].Tag, y=tags[:50].counts)

plt.xticks(rotation=90)
plt.ylabel("Number of questions")

In [ ]:
tags.info()

In [ ]:
tags.describe()

## 2.2. Tags data grouped by questions

In [ ]:
questions_tags = tags_data.Id.value_counts().rename_axis('Id').reset_index(name='counts')

In [ ]:
questions_tags.shape

In [ ]:
questions_tags.info()

In [ ]:
questions_tags.describe()

In [ ]:
questions_tags.head()

In [ ]:
plt.figure(figsize=(16,6))

plt.title("Tags Distribution According To Questions")
sns.barplot(x=questions_tags.loc[:50].Id, y=questions_tags[:50].counts)

plt.xticks(rotation=90)
plt.ylabel("Number of tags")

In [ ]:
for i in range(5):
    a = i+1
    df = questions_tags.loc[questions_tags['counts']==a]
    percent = (df.shape[0]/questions_tags.shape[0])*100
    print(percent, "% of questions which means: ", df.shape[0]," questions have ",a, " tags")

In [ ]:
questions_ids_with_one_tag = questions_tags.loc[questions_tags['counts']==1].Id
mask_questions_one_tag = questions_data["Id"].isin(questions_ids_with_one_tag)
questions_data_one_tag = questions_data.loc[mask_questions_one_tag]
answers_data_one_tag = answers_data["ParentId"].isin(questions_ids_with_one_tag)
answers_data_one_tag = answers_data.loc[answers_data_one_tag]

In [ ]:
answers_data_one_tag.shape

In [ ]:
questions_data_one_tag.shape

In [ ]:
questions_data_one_tag.columns

In [ ]:
answers_data_one_tag.columns

In [ ]:
qs = questions_data_one_tag.drop(['CreationDate', 'ClosedDate', 'OwnerUserId'], axis=1)

In [ ]:
answ = answers_data_one_tag.drop(['CreationDate', 'OwnerUserId'],axis=1)

In [ ]:
qs.to_csv('./qs')
answ.to_csv('./answ')

## 2.3. Answers data

In [ ]:
answers_data.head()

In [ ]:
answers_data.info()

In [ ]:
answers_data.isnull().sum()

In [ ]:
answers = answers_data[['Id', 'ParentId', 'Score']]

In [ ]:
answers.info()

In [ ]:
questions_grouped_by_answers = answers.ParentId.value_counts().rename_axis('ParentId').reset_index(name='counts').sort_values(by=['counts'], ascending=False)

In [ ]:
questions_grouped_by_answers.info()

In [ ]:
questions_grouped_by_answers.describe()

In [ ]:
questions_grouped_by_answers.head(10)

In [ ]:
plt.figure(figsize=(16,6))

plt.title("Answers Distribution According To Questions")
#sns.barplot(x=questions_grouped_by_answers.loc[:50].ParentId, y=questions_grouped_by_answers[:50].counts)
sns.barplot(x='ParentId', y="counts", 
            data=questions_grouped_by_answers.loc[:50],
            order=questions_grouped_by_answers.loc[:50].sort_values('counts', ascending=False).ParentId)

plt.xticks(rotation=90)
plt.ylabel("Number of Answers")

In [ ]:
questions_grouped_by_answers.loc[questions_grouped_by_answers['ParentId']==517970]

In [ ]:
answers_data.loc[answers_data['ParentId']==517970].head()

In [ ]:
answers.Score.describe()

In [ ]:
df = answers.sort_values('Score', ascending=False)
d = df.head(50)
d.shape

In [ ]:
plt.figure(figsize=(16,6))

plt.title("Score")
sns.barplot(x='Id', y="Score", 
            data=d,
            order=d.sort_values('Score', ascending=False).Id)

plt.xticks(rotation=90)
plt.ylabel("Number of Answers")

# 3. Data cleaning

In [ ]:
question = questions_data.loc[0]
question

In [ ]:
def tags_in_question(question):
    df = tags_data.loc[tags_data["Id"]==question.Id]
    result = []
    for index, row in df.iterrows():
        result.append(row.Tag)
    return result

In [ ]:
def question_to_tags_vector(question):
    tags_q = tags_in_question(question)
    result = pd.Series()
    for index, row in tags.iterrows():
        if row.Tag in tags_q:
            result[row.Tag]=1
        else:
            result[row.Tag]=0
    return result

In [ ]:
question_to_tags_vector(question)

# 4. Data Extraction

In [ ]:
questions_data['CreationDate'].unique()

In [ ]:
train_questions = questions_data[(questions_data['CreationDate'] >= '2008-01-01')&(questions_data['CreationDate'] <= '2008-12-31')]
test_questions = questions_data[(questions_data['CreationDate'] >= '2009-01-01')&(questions_data['CreationDate'] <= '2009-12-31')]

In [ ]:
print(train_questions.shape)
print(test_questions.shape)

In [ ]:
def questions_year(year):
    start_date = str(year)+'-01-01'
    end_date = str(year)+'-12-31'
    data = questions_data[(questions_data['CreationDate'] >= start_date)&(questions_data['CreationDate'] <= end_date)]
    print("Dataset size from ", start_date, " to ", end_date,": ",data.shape)
    return data

In [ ]:
for i in range(2008, 2017):
    a = questions_year(i)

In [ ]:
questions_2009 = questions_year(2009)
questions_2008 = questions_year(2008)

In [ ]:
ids = questions_2008.Id.unique()
ids = ids.tolist()

In [ ]:
answers_2008_mask = answers_data['ParentId'].isin(ids)
answers_2008 = answers_data.loc[answers_2008_mask]
answers_2008.shape

In [ ]:
ids_2009 = questions_2009.Id.unique()
ids_2009 = ids_2009.tolist()

answers_2009_mask = answers_data['ParentId'].isin(ids_2009)
answers_2009 = answers_data.loc[answers_2009_mask]
answers_2009.shape

In [ ]:
questions_2008.to_csv('./questions_2008.csv')
questions_2009.to_csv('./questions_2009.csv')

answers_2008.to_csv('./answers_2008.csv')
answers_2009.to_csv('./answers_2009.csv')